<a href="https://colab.research.google.com/github/joshjarvey/Coursera_Capstone/blob/master/Capstone_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
import requests

In [4]:
  #This data was gathered from an open public source via the City of Madison's Database:http://data-cityofmadison.opendata.arcgis.com/datasets/neighborhood-plans
  #Since the database is an ARCGIS database, it only contained SHAPE data.
  #The data was downloaded as a "SHAPE" file and converted to latitude and longitude using the free tool here:https://mygeodata.cloud/converter/shp-to-latlong

  #read in the .csv data, and drop the columns that are not needed for the analysis of this project
df_msn_neighborhood = pd.read_csv('/tmp/Neighborhood_Plans.csv').drop(['OBJECTID_1','OBJECTID','Status','Year','SHAPE_Leng','Link','ShapeSTAre','ShapeSTLen'], axis=1)

  #rename the columns for easier referencing in the project.
df_msn_neighborhood.rename(columns={'Y':'Latitude','X':'Longitude','PlanName':'Neighborhood'}, inplace=True)

  #cleaning the names of the neighborhood values 
df_msn_neighborhood["Neighborhood"]= df_msn_neighborhood["Neighborhood"].str.replace('Neighborhood', '')
df_msn_neighborhood["Neighborhood"]= df_msn_neighborhood["Neighborhood"].str.replace('Plan', '')
df_msn_neighborhood


,Longitude,Latitude,Neighborhood
0,-89.341014,43.098683,Schenk-Atwood-Starkweather-Worthington Park
1,-89.372108,43.086926,Tenney-Lapham-Old Market
2,-89.358018,43.087919,Marquette-Schenk-Atwood
3,-89.409940,43.061988,Brittingham-Vilas
4,-89.390866,43.054009,Bay Creek
5,-89.346647,43.048048,Broadway-Simpson-Waunona
6,-89.456386,43.027492,Allied-Dunn's Marsh
7,-89.329012,43.113940,Carpenter-Ridgeway-Hawthorne-Truax
8,-89.488393,43.045119,Southwest
9,-89.384803,43.051256,South Madison


In [5]:
#getting the lat/long coordinates of toronto, using the geolocater module
address = 'Madison, WI'

geolocator = Nominatim(user_agent="madison_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madison are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madison are 43.074761, -89.3837613.


In [6]:
#create map of Toronto using latitude and longitude values
map_madison = folium.Map(location=[latitude, longitude], zoom_start=13)

    #add markers to map
for lat, lng, label in zip(df_msn_neighborhood['Latitude'], df_msn_neighborhood['Longitude'], df_msn_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madison)  

    #display map
map_madison

In [0]:
#defining foursquare credentials
CLIENT_ID = 'RNBGL5BEPF5JT2ZT0KK4KHBEFTH0LRVTDAC5KQNCCTQZTBZU' # your Foursquare ID
CLIENT_SECRET = '41IV3LTCYNOFHJMTYFAPZC0UI3AYLWVYP41IMXQDF0NMGNVQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [8]:
#reviewing lat/long of the first item in the new dataframe
neighborhood_latitude = df_msn_neighborhood.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_msn_neighborhood.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_msn_neighborhood.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Tenney-Lapham-Old Market   are 43.0869264952734, -89.3721077659885.


In [14]:
#compiling the URL that we'll pass to the foursquare API 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1610 # define radius
query = 'Grocery Store'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    query
    )

print (url)

https://api.foursquare.com/v2/venues/explore?&client_id=RNBGL5BEPF5JT2ZT0KK4KHBEFTH0LRVTDAC5KQNCCTQZTBZU&client_secret=41IV3LTCYNOFHJMTYFAPZC0UI3AYLWVYP41IMXQDF0NMGNVQ&v=20180605&ll=43.0869264952734,-89.3721077659885&radius=1610&limit=100&query=Grocery Store


In [0]:
#calling the foursquare API with the compiled URL. It returns a .json file with all the data, so storing that .json into a variable
results = requests.get(url).json()

In [11]:
results

{'meta': {'code': 200, 'requestId': '5dcdb626b9a389001b9bd84e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5646c59f498eb7f011b73aeb-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '5646c59f498eb7f011b73aeb',
       'location': {'address': '810 E. Washington Ave.',
        'cc': 'US',
        'city': 'Madison',
        'country': 'United States',
        'distance': 642,
        'formattedAddress': ['810 E. Washington Ave.',
         'Madison, WI 53703',
         'United States'],
        'labeledLatLngs': [{'label': 'dis

In [61]:
type = results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']
address = results['response']['groups'][0]['items'][0]['venue']['location']['formattedAddress'][0] + results['response']['groups'][0]['items'][0]['venue']['location']['formattedAddress'][1]

store_lat = results['response']['groups'][0]['items'][0]['venue']['location']['labeledLatLngs'][0]['lat']
store_lng = results['response']['groups'][0]['items'][0]['venue']['location']['labeledLatLngs'][0]['lng']

store_name = store_lng = results['response']['groups'][0]['items'][0]['venue']['name']

store_name
grocery_Stores = [type,store_name,address,store_lat,store_lng]
grocery_Stores

['Grocery Store',
 'Festival Foods',
 '810 E. Washington Ave.Madison, WI 53703',
 43.08160876418695,
 'Festival Foods']